In [ ]:
import sys
sys.path.append('/gpfs/exfel/exp/SPB/202501/p006933/usr/Software/analysistools')
import data_helper as dh
import focus_scan as fs

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from pathlib import Path
from scipy.interpolate import interp1d

path = dh.expPath+'Results/FocusScans/Data/'
directory=Path(path)

runs = np.arange(278, 299)
format_runs = [dh.run_format(r) for r in runs]
#print(format_runs)
#279-283,289
values_to_remove = {279, 280,281,282,283,289}
runs = runs[~np.isin(runs, list(values_to_remove))]


files = [file.name for file in sorted(directory.glob('fyield_*n300.h5'), reverse=False) if int(file.name.split('_')[1][1:5]) in runs]

nshot=files[0].split('_')[5][1:4]

f_yield = []
for file in files:
    df = pd.read_hdf(path+file, ignore_index=True)
    f_yield.append(df)

df = pd.concat(f_yield, ignore_index=True)
print(df.columns)

df_sorted=df.sort_values(by='transmission')
threshold=1.06
# Listen zur Speicherung der gruppierten Daten
grouped_transmissions = []
grouped_f = []
grouped_photon_energy = []

# Initialisiere die erste Gruppe mit dem ersten Eintrag
current_group_trans = [df_sorted.iloc[0]['transmission']]
current_group_f = [df_sorted.iloc[0]['f_yield']]
current_group_photon_energy = [df_sorted.iloc[0]['photon_energy']]

# Iteriere über alle Zeilen (ab dem zweiten Eintrag)
for i in range(1, len(df_sorted)):
    # Vergleiche den aktuellen Transmission-Wert mit dem letzten Wert der aktuellen Gruppe
    if abs(df_sorted.iloc[i]['transmission'] - current_group_trans[-1]) < threshold:
        # Wenn die Differenz innerhalb des Schwellwerts liegt, füge alle Werte der aktuellen Gruppe hinzu
        current_group_trans.append(df_sorted.iloc[i]['transmission'])
        current_group_f.append(df_sorted.iloc[i]['f_yield'])
        current_group_photon_energy.append(df_sorted.iloc[i]['photon_energy'])
    else:
        # Wenn die Transmission zu weit entfernt ist, speichere die bisherige Gruppe und starte eine neue
        grouped_transmissions.append(current_group_trans)
        grouped_f.append(current_group_f)
        grouped_photon_energy.append(current_group_photon_energy)
        
        current_group_trans = [df_sorted.iloc[i]['transmission']]
        current_group_f = [df_sorted.iloc[i]['f_yield']]
        current_group_photon_energy = [df_sorted.iloc[i]['photon_energy']]

# Füge die letzte Gruppe hinzu
grouped_transmissions.append(current_group_trans)
grouped_f.append(current_group_f)
grouped_photon_energy.append(current_group_photon_energy)

# Erstelle einen DataFrame mit den gruppierten Daten
grouped_df = pd.DataFrame({
    'Grouped transmissions': grouped_transmissions,
    'fyield': grouped_f,
    'photon_energy': grouped_photon_energy
})

for index in range(len(grouped_df['fyield'])):
    print(grouped_df['Grouped transmissions'][index])
    plt.plot(grouped_df['photon_energy'][index],grouped_df['fyield'][index], ls='', marker='o')
    plt.title(f'transmission {round(np.mean(grouped_df["Grouped transmissions"][index]), 2)}')
    plt.xlabel('photon energy in eV')
    plt.ylabel('fyield in arb. u.')
    plt.savefig(f'/gpfs/exfel/exp/SPB/202501/p006933/usr/Shared/muthreich/fyield_{round(np.mean(grouped_df["Grouped transmissions"][index]), 2)}.png', format='png')
    plt.show()


In [ ]:

runs_byp = np.arange(167, 223)
values_to_remove = {192}
runs_byp = runs_byp[~np.isin(runs_byp, list(values_to_remove))]
runs_10nm=np.arange(278, 305)
values_to_remove = {279, 280,281,282,283,289}
runs_10nm = runs_10nm[~np.isin(runs_10nm, list(values_to_remove))]

runs = np.arange(278, 299)


def get_grouped_df(runs, threshold=0.06, nshots=300):
    files = [file.name for file in sorted(directory.glob(f'fyield_*n{nshots}.h5'), reverse=False) if int(file.name.split('_')[1][1:5]) in runs]
    
    nshot=files[0].split('_')[5][1:4]
    
    f_yield = []
    for file in files:
        df = pd.read_hdf(path+file, ignore_index=True)
        f_yield.append(df)
    
    df = pd.concat(f_yield, ignore_index=True)
    print(df.columns)
    
    df_sorted=df.sort_values(by='transmission')
    
    # Listen zur Speicherung der gruppierten Daten
    grouped_transmissions = []
    grouped_f = []
    grouped_photon_energy = []
    
    # Initialisiere die erste Gruppe mit dem ersten Eintrag
    current_group_trans = [df_sorted.iloc[0]['transmission']]
    current_group_f = [df_sorted.iloc[0]['f_yield']]
    current_group_photon_energy = [df_sorted.iloc[0]['photon_energy']]
    
    # Iteriere über alle Zeilen (ab dem zweiten Eintrag)
    for i in range(1, len(df_sorted)):
        # Vergleiche den aktuellen Transmission-Wert mit dem letzten Wert der aktuellen Gruppe
        if abs(df_sorted.iloc[i]['transmission'] - current_group_trans[-1]) < threshold:
            # Wenn die Differenz innerhalb des Schwellwerts liegt, füge alle Werte der aktuellen Gruppe hinzu
            current_group_trans.append(df_sorted.iloc[i]['transmission'])
            current_group_f.append(df_sorted.iloc[i]['f_yield'])
            current_group_photon_energy.append(df_sorted.iloc[i]['photon_energy'])
        else:
            # Wenn die Transmission zu weit entfernt ist, speichere die bisherige Gruppe und starte eine neue
            grouped_transmissions.append(current_group_trans)
            grouped_f.append(current_group_f)
            grouped_photon_energy.append(current_group_photon_energy)
            
            current_group_trans = [df_sorted.iloc[i]['transmission']]
            current_group_f = [df_sorted.iloc[i]['f_yield']]
            current_group_photon_energy = [df_sorted.iloc[i]['photon_energy']]
    
    # Füge die letzte Gruppe hinzu
    grouped_transmissions.append(current_group_trans)
    grouped_f.append(current_group_f)
    grouped_photon_energy.append(current_group_photon_energy)
    
    # Erstelle einen DataFrame mit den gruppierten Daten
    grouped_df = pd.DataFrame({
        'Grouped transmissions': grouped_transmissions,
        'fyield': grouped_f,
        'photon_energy': grouped_photon_energy
    })
    return grouped_df

grouped_df_byp=get_grouped_df(runs_byp, threshold=0.1)
grouped_df_10nm=get_grouped_df(runs_10nm, threshold=1, nshots=1000)
grouped_df=grouped_df_byp
for index in range(len(grouped_df['fyield'])):
    print(grouped_df['Grouped transmissions'][index])
    plt.plot(grouped_df['photon_energy'][index],grouped_df['fyield'][index], ls='', marker='o')
    plt.title(f'transmission {round(np.mean(grouped_df["Grouped transmissions"][index]), 2)}')
    plt.xlabel('photon energy in eV')
    plt.ylabel('fyield in arb. u.')
    plt.savefig(f'/gpfs/exfel/exp/SPB/202501/p006933/usr/Shared/muthreich/fyield_{round(np.mean(grouped_df["Grouped transmissions"][index]), 2)}.png', format='png')
    plt.show()

plt.plot(grouped_df_byp['photon_energy'][3],grouped_df_byp['fyield'][3], ls='', marker='o', label='bypyramids')
plt.plot(grouped_df_10nm['photon_energy'][0],grouped_df_10nm['fyield'][0], ls='', marker='o', label='10nm')

plt.xlabel('photon energy in eV')
plt.ylabel('fyield in arb. u.')
#plt.title(f'transmission {round(np.mean(grouped_df["Grouped transmissions"][index]), 2)}')
#plt.savefig(f'/gpfs/exfel/exp/SPB/202501/p006933/usr/Shared/muthreich/fyield_{round(np.mean(grouped_df["Grouped transmissions"][index]), 2)}.png', format='png')
plt.show()
